In [1]:
import pandas as pd
import numpy as np

# 1

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.
water.txt

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [2]:
data=pd.read_table('water.txt')
data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [3]:
round(data.iloc[:,2:].corr(method='pearson'),4)

,mortality,hardness
mortality,1.0000,-0.6548
hardness,-0.6548,1.0000


# 2
В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки. 

In [4]:
round(data.iloc[:,2:].corr(method='spearman'),4)

,mortality,hardness
mortality,1.0000,-0.6317
hardness,-0.6317,1.0000


# 3
Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [5]:
south=data.iloc[:,2:].loc[data.location=='South']
north=data.iloc[:,2:].loc[data.location=='North']

In [6]:
north_corr=round(north.corr(method='pearson'),4)
north_corr

,mortality,hardness
mortality,1.0000,-0.3686
hardness,-0.3686,1.0000


In [7]:
south_corr=round(south.corr(method='pearson'),4)
south_corr

,mortality,hardness
mortality,1.0000,-0.6022
hardness,-0.6022,1.0000


# 4

Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.  

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

пол: 0-ж/1-м
частота: 0-реже раза в месяц, 1-чаще

In [8]:
a=718
d=239
b=203
c=515
mcc=(a*d-b*c)/np.sqrt((a+b)*(a+c)*(b+d)*(c+d))
round(mcc,4)

0.109

# 5

В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. 

$H_0\colon$ mcc=0

$H_1\colon$ mcc!=0

In [9]:
n=239+203+718+515
t=n*mcc**2
t

19.90154208981099

In [10]:
from scipy.stats import chi2,chi2_contingency

In [11]:
round(2*(1-chi2.cdf(abs(t),1)),5)

2e-05

In [12]:
g, p, dof, expctd=chi2_contingency([[718,203],[515,239]])
round(p,5)

1e-05

In [13]:
g, p, dof, expctd=chi2_contingency([[515,239],[718,203]])
round(p,5)

1e-05

# 6
В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

In [80]:
import scipy

In [81]:
def proportions_diff_confint_ind(p1, p2,n1,n2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

In [82]:
round(proportions_diff_confint_ind(239/(239+515),203/(718+203),515+239,718+203)[0],4)

0.0539

# 7
Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили 5.5×10−85.5\times10^{-8}5.5×10−8, нужно ввести 8).

$H_0\colon$ p1=p2

$H_1\colon$ p1!=p2

In [66]:
def proportions_diff_z_stat_ind(p1, p2,n1,n2):
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [67]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [83]:
proportions_diff_z_test(proportions_diff_z_stat_ind(239/(239+515),203/(718+203),515+239,718+203))

8.153453089576601e-06

=> нулевая гипотеза отвергается, доли среди мужчин и женщин не равны

# 8

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"
	Не доволен	Более или менее	Доволен
Не очень счастлив	197	111	33
Достаточно счастлив	382	685	331
Очень счастлив	110	342	333

Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [72]:
g, p, dof, expctd=chi2_contingency([[197,111,33],[382,685,331],[110,342,333]])
round(g,4)

293.6831

# 9

На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−85.5\times10^{-8}5.5×10−8, нужно ввести 8).

In [73]:
p

2.4964299580093467e-62

# 10

Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки. 

In [89]:
n=sum(np.array([197,111,33,382,685,331,110,342,333]))
fi = np.sqrt(g/(n*(3-1)))
round(fi,4)

0.2412